In [6]:
import torch

In [10]:
# print the version of PyTorch and the available devices
print("PyTorch version: ", torch.__version__)

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
    print("Number of CUDA devices: ", torch.cuda.device_count())
    print("Current CUDA device: ", torch.cuda.current_device())
    print("Name of current CUDA device: ", torch.cuda.get_device_name(torch.cuda.current_device()))

PyTorch version:  2.2.2+cu121
CUDA is available.
Number of CUDA devices:  1
Current CUDA device:  0
Name of current CUDA device:  NVIDIA GeForce RTX 3050 Laptop GPU
